In [1]:
import sys
sys.path.insert(0, '..')
import zarr
import numpy as np
np.random.seed(42)
import cProfile

## Demonstrate advanced indexing

### Indexing with Boolean arrays

In [2]:
a = np.arange(10)

In [3]:
ix = np.random.binomial(1, 0.5, size=a.shape[0]).astype(bool)
ix

array([False,  True,  True,  True, False, False, False,  True,  True,  True], dtype=bool)

In [4]:
a[ix]

array([1, 2, 3, 7, 8, 9])

In [5]:
za = zarr.array(a, chunks=2)
za[ix]

array([1, 2, 3, 7, 8, 9])

In [6]:
za[ix] = a[ix] * 10
za[:]

array([ 0, 10, 20, 30,  4,  5,  6, 70, 80, 90])

### Indexing with integer arrays

In [7]:
ix = np.random.choice(a.shape[0], size=a.shape[0]//2)
ix.sort()  # only monotonically increasing indices are supported
ix

array([1, 4, 5, 5, 7])

In [8]:
a[ix]

array([1, 4, 5, 5, 7])

In [9]:
za = zarr.array(a, chunks=2)
za[ix]

array([1, 4, 5, 5, 7])

In [10]:
za[ix] = a[ix] * 10
za[:]

array([ 0, 10,  2,  3, 40, 50,  6, 70,  8,  9])

### Multidimensional indexing

N.B., orthogonaly indexing is implemented. This is different from numpy fancy indexing if more than one dimension is indexed with an array.

In [11]:
b = np.arange(100).reshape(10, 10)

In [12]:
ix0 = np.random.binomial(1, 0.5, size=b.shape[0]).astype(bool)
ix0

array([False,  True,  True,  True, False, False,  True, False, False,  True], dtype=bool)

In [13]:
ix1 = np.random.binomial(1, 0.5, size=b.shape[1]).astype(bool)
ix1

array([False, False,  True, False,  True, False,  True,  True, False, False], dtype=bool)

In [14]:
b[np.ix_(ix0, ix1)]

array([[12, 14, 16, 17],
       [22, 24, 26, 27],
       [32, 34, 36, 37],
       [62, 64, 66, 67],
       [92, 94, 96, 97]])

In [15]:
zb = zarr.array(b, chunks=(2, 2))
zb[ix0, ix1]

array([[12, 14, 16, 17],
       [22, 24, 26, 27],
       [32, 34, 36, 37],
       [62, 64, 66, 67],
       [92, 94, 96, 97]])

In [16]:
zb[ix0, ix1] = -1
zb[:]

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, -1, 13, -1, 15, -1, -1, 18, 19],
       [20, 21, -1, 23, -1, 25, -1, -1, 28, 29],
       [30, 31, -1, 33, -1, 35, -1, -1, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [60, 61, -1, 63, -1, 65, -1, -1, 68, 69],
       [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
       [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
       [90, 91, -1, 93, -1, 95, -1, -1, 98, 99]])

In [17]:
ix0 = np.random.choice(b.shape[0], size=b.shape[0]//2)
ix0.sort()  # only monotonically increasing indices are supported
ix0

array([1, 8, 8, 9, 9])

In [18]:
ix1 = np.random.choice(b.shape[1], size=b.shape[1]//2)
ix1.sort()  # only monotonically increasing indices are supported
ix1

array([1, 3, 4, 6, 7])

In [19]:
b[np.ix_(ix0, ix1)]

array([[11, 13, 14, 16, 17],
       [81, 83, 84, 86, 87],
       [81, 83, 84, 86, 87],
       [91, 93, 94, 96, 97],
       [91, 93, 94, 96, 97]])

In [20]:
zb = zarr.array(b, chunks=(2, 2))
zb[ix0, ix1]

array([[11, 13, 14, 16, 17],
       [81, 83, 84, 86, 87],
       [81, 83, 84, 86, 87],
       [91, 93, 94, 96, 97],
       [91, 93, 94, 96, 97]])

In [21]:
zb[ix0, ix1] = -1
zb[:]

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, -1, 12, -1, -1, 15, -1, -1, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
       [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
       [80, -1, 82, -1, -1, 85, -1, -1, 88, 89],
       [90, -1, 92, -1, -1, 95, -1, -1, 98, 99]])

### Indexing with zarr bool arrays

In [22]:
ix = np.random.binomial(1, 0.5, size=a.shape[0]).astype(bool)
zix = zarr.array(ix, chunks=2)

In [23]:
za = zarr.array(a, chunks=2)
za[ix]

array([1, 3, 5, 6, 8, 9])

In [24]:
# will not load all zix into memory
za[zix]

array([1, 3, 5, 6, 8, 9])

## Benchmarking

In [25]:
c = np.arange(100000000)
c.nbytes

800000000

In [26]:
zc = zarr.array(c)
zc.info

Type,zarr.core.Array
Data type,int64
Shape,"(100000000,)"
Chunk shape,"(48829,)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,builtins.dict
No. bytes,800000000 (762.9M)
No. bytes stored,11870277 (11.3M)
Storage ratio,67.4


In [27]:
%time c.copy()

CPU times: user 148 ms, sys: 52 ms, total: 200 ms
Wall time: 200 ms


array([       0,        1,        2, ..., 99999997, 99999998, 99999999])

In [28]:
%time zc[:]

CPU times: user 480 ms, sys: 420 ms, total: 900 ms
Wall time: 308 ms


array([       0,        1,        2, ..., 99999997, 99999998, 99999999])

### bool dense selection

In [29]:
# relatively dense selection
ix_dense_bool = np.random.binomial(1, 0.5, size=c.shape[0]).astype(bool)
np.count_nonzero(ix_dense_bool)

49994863

In [30]:
%time c[ix_dense_bool]

CPU times: user 612 ms, sys: 20 ms, total: 632 ms
Wall time: 628 ms


array([       0,        1,        2, ..., 99999994, 99999995, 99999996])

In [31]:
%time zc[ix_dense_bool]

CPU times: user 1.5 s, sys: 208 ms, total: 1.71 s
Wall time: 983 ms


array([       0,        1,        2, ..., 99999994, 99999995, 99999996])

In [32]:
cProfile.run('zc[ix_dense_bool]', sort='time')

         73776 function calls in 1.005 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2048    0.666    0.000    0.967    0.000 core.py:679(_chunk_getitem)
     2048    0.255    0.000    0.274    0.000 core.py:839(_decode_chunk)
     2048    0.010    0.000    0.019    0.000 util.py:418(get_chunk_selections)
     2048    0.009    0.000    0.009    0.000 {built-in method numpy.core.multiarray.count_nonzero}
     2048    0.008    0.000    0.008    0.000 {built-in method numpy.core.multiarray.frombuffer}
     2048    0.008    0.000    0.008    0.000 {method 'reshape' of 'numpy.ndarray' objects}
        1    0.006    0.006    1.004    1.004 core.py:484(_getitem_nd)
     2048    0.005    0.000    0.009    0.000 util.py:114(is_total_slice)
     2048    0.005    0.000    0.010    0.000 arrayprint.py:381(wrapper)
     2048    0.004    0.000    0.016    0.000 {method 'join' of 'str' objects}
     2048    0.004    0.000    0.004 

### int dense selection

In [33]:
ix_dense_int = np.nonzero(ix_dense_bool)[0]
len(ix_dense_int)

49994863

In [34]:
%time c[ix_dense_int]

CPU times: user 144 ms, sys: 20 ms, total: 164 ms
Wall time: 160 ms


array([       0,        1,        2, ..., 99999994, 99999995, 99999996])

In [35]:
%time zc[ix_dense_int]

CPU times: user 1.3 s, sys: 152 ms, total: 1.45 s
Wall time: 1.16 s


array([       0,        1,        2, ..., 99999994, 99999995, 99999996])

In [36]:
cProfile.run('zc[ix_dense_int]', sort='time')

         71758 function calls in 1.208 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.662    0.662    0.864    0.864 util.py:191(__init__)
     2048    0.129    0.000    0.139    0.000 core.py:839(_decode_chunk)
        1    0.119    0.119    0.119    0.119 {built-in method numpy.core.multiarray.bincount}
     2048    0.116    0.000    0.278    0.000 core.py:679(_chunk_getitem)
        1    0.063    0.063    0.063    0.063 function_base.py:1848(diff)
     2048    0.042    0.000    0.045    0.000 util.py:225(get_chunk_sel)
        4    0.020    0.005    0.020    0.005 {method 'reduce' of 'numpy.ufunc' objects}
     2048    0.010    0.000    0.059    0.000 util.py:418(get_chunk_selections)
        1    0.006    0.006    1.207    1.207 core.py:484(_getitem_nd)
     4096    0.005    0.000    0.005    0.000 util.py:235(get_out_sel)
     2048    0.005    0.000    0.007    0.000 util.py:114(is_total_slice)
     204

### bool sparse selection

In [37]:
# relatively sparse selection
ix_sparse_bool = np.random.binomial(1, 0.0001, size=c.shape[0]).astype(bool)
np.count_nonzero(ix_sparse_bool)

9950

In [38]:
%time c[ix_sparse_bool]

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 17.8 ms


array([   12643,    15188,    16392, ..., 99989960, 99995101, 99999097])

In [39]:
%time zc[ix_sparse_bool]

CPU times: user 472 ms, sys: 88 ms, total: 560 ms
Wall time: 262 ms


array([   12643,    15188,    16392, ..., 99989960, 99995101, 99999097])

In [40]:
cProfile.run('zc[ix_sparse_bool]', sort='time')

         73436 function calls in 0.289 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2038    0.172    0.000    0.184    0.000 core.py:839(_decode_chunk)
     2038    0.035    0.000    0.248    0.000 core.py:679(_chunk_getitem)
     2048    0.013    0.000    0.013    0.000 {built-in method numpy.core.multiarray.count_nonzero}
     2038    0.011    0.000    0.020    0.000 util.py:418(get_chunk_selections)
        1    0.006    0.006    0.289    0.289 core.py:484(_getitem_nd)
     2038    0.006    0.000    0.009    0.000 util.py:114(is_total_slice)
     2038    0.005    0.000    0.011    0.000 arrayprint.py:381(wrapper)
     2038    0.005    0.000    0.005    0.000 {method 'reshape' of 'numpy.ndarray' objects}
     2038    0.005    0.000    0.017    0.000 {method 'join' of 'str' objects}
     2038    0.004    0.000    0.004    0.000 {built-in method numpy.core.multiarray.frombuffer}
     2038    0.004    0.000    0.004 

### int sparse selection

In [41]:
ix_sparse_int = np.nonzero(ix_sparse_bool)[0]
len(ix_sparse_int)

9950

In [42]:
%time c[ix_sparse_int]

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 169 µs


array([   12643,    15188,    16392, ..., 99989960, 99995101, 99999097])

In [43]:
%time zc[ix_sparse_int]

CPU times: user 504 ms, sys: 68 ms, total: 572 ms
Wall time: 262 ms


array([   12643,    15188,    16392, ..., 99989960, 99995101, 99999097])

In [44]:
cProfile.run('zc[ix_sparse_int]', sort='time')

         71408 function calls in 0.241 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2038    0.158    0.000    0.169    0.000 core.py:839(_decode_chunk)
     2038    0.011    0.000    0.014    0.000 util.py:225(get_chunk_sel)
     2038    0.010    0.000    0.028    0.000 util.py:418(get_chunk_selections)
     2038    0.010    0.000    0.207    0.000 core.py:679(_chunk_getitem)
        1    0.006    0.006    0.241    0.241 core.py:484(_getitem_nd)
     2038    0.005    0.000    0.009    0.000 util.py:114(is_total_slice)
     4076    0.005    0.000    0.005    0.000 util.py:235(get_out_sel)
     2038    0.005    0.000    0.010    0.000 arrayprint.py:381(wrapper)
     2038    0.005    0.000    0.005    0.000 {method 'reshape' of 'numpy.ndarray' objects}
     2038    0.004    0.000    0.016    0.000 {method 'join' of 'str' objects}
     2038    0.003    0.000    0.003    0.000 {built-in method numpy.core.multiarray.fromb

### sparse bool selection as zarr array

In [45]:
zix_sparse_bool = zarr.array(ix_sparse_bool)
zix_sparse_bool.info

Type,zarr.core.Array
Data type,bool
Shape,"(100000000,)"
Chunk shape,"(195313,)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,builtins.dict
No. bytes,100000000 (95.4M)
No. bytes stored,511163 (499.2K)
Storage ratio,195.6


In [46]:
%time zc[zix_sparse_bool]

CPU times: user 932 ms, sys: 180 ms, total: 1.11 s
Wall time: 570 ms


array([   12643,    15188,    16392, ..., 99989960, 99995101, 99999097])

### h5py comparison

N.B., not really fair because using slower compressor, but for interest...

In [47]:
import h5py
import tempfile

In [48]:
h5f = h5py.File(tempfile.mktemp(), driver='core', backing_store=False)

In [49]:
hc = h5f.create_dataset('c', data=c, compression='gzip', compression_opts=1, chunks=zc.chunks, shuffle=True)
hc

<HDF5 dataset "c": shape (100000000,), type "<i8">

In [50]:
%time hc[:]

CPU times: user 1.14 s, sys: 40 ms, total: 1.18 s
Wall time: 1.17 s


array([       0,        1,        2, ..., 99999997, 99999998, 99999999])

In [ ]:
%time hc[ix_sparse_bool]

CPU times: user 1.1 s, sys: 0 ns, total: 1.1 s
Wall time: 1.1 s


array([   12643,    15188,    16392, ..., 99989960, 99995101, 99999097])

In [ ]:
# this is pathological, takes > 1 minute 
%time hc[ix_dense_bool]